### homework1：练习groupby

In [1]:
import numpy as np
import pandas as pd

In [2]:
#分组后给名称加前缀
dict_obj = {'key1': ['a', 'b', 'a', 'b',
                     'a', 'b', 'a', 'a'],
            'key2': ['one', 'one', 'two', 'three',
                     'two', 'two', 'one', 'three'],
            'data1': np.random.randint(1, 10, 8),
            'data2': np.random.randint(1, 10, 8)}
df_obj = pd.DataFrame(dict_obj)
print(df_obj)

  key1   key2  data1  data2
0    a    one      2      3
1    b    one      4      9
2    a    two      3      6
3    b  three      9      7
4    a    two      7      1
5    b    two      2      3
6    a    one      7      6
7    a  three      5      8


In [3]:
df_obj.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   key1    8 non-null      object
 1   key2    8 non-null      object
 2   data1   8 non-null      int32 
 3   data2   8 non-null      int32 
dtypes: int32(2), object(2)
memory usage: 324.0+ bytes


In [4]:
# 按key1分组后，计算data1，data2的统计信息并附加到原始表格中，并添加表头前缀
k1_sum = df_obj.groupby('key1').mean(numeric_only=True).add_prefix('mean_')  # 使用 numeric_only=True 参数可以确保只对数值列进行计算
print(k1_sum)

      mean_data1  mean_data2
key1                        
a            4.8    4.800000
b            5.0    6.333333


In [5]:
# 方法2，使用transform，分组后计算结果和原本的df保持一致
k1_sum_tf = df_obj.loc[:, ['key1', 'data1', 'data2']].groupby('key1').transform('mean').add_prefix('mean_')
k1_sum_tf

,mean_data1,mean_data2
0,4.8,4.800000
1,5.0,6.333333
2,4.8,4.800000
3,5.0,6.333333
4,4.8,4.800000
5,5.0,6.333333
6,4.8,4.800000
7,4.8,4.800000


In [13]:
df_obj[k1_sum_tf.columns] = k1_sum_tf
print(df_obj)

  key1   key2  data1  data2  mean_data1  mean_data2
0    a    one      9      6         4.2    6.000000
1    b    one      2      5         4.0    2.333333
2    a    two      5      2         4.2    6.000000
3    b  three      8      9         4.0    2.333333
4    a    two      3      9         4.2    6.000000
5    b    two      2      5         4.0    2.333333
6    a    one      3      8         4.2    6.000000
7    a  three      1      1         4.2    6.000000


In [6]:
del df_obj['key2']
df_obj

,key1,data1,data2
0,a,2,3
1,b,4,9
2,a,3,6
3,b,9,7
4,a,7,1
5,b,2,3
6,a,7,6
7,a,5,8


In [7]:
df_obj.groupby('key1').transform('mean').add_prefix('mean_')

,mean_data1,mean_data2
0,4.8,4.800000
1,5.0,6.333333
2,4.8,4.800000
3,5.0,6.333333
4,4.8,4.800000
5,5.0,6.333333
6,4.8,4.800000
7,4.8,4.800000


In [8]:
df_obj

,key1,data1,data2
0,a,2,3
1,b,4,9
2,a,3,6
3,b,9,7
4,a,7,1
5,b,2,3
6,a,7,6
7,a,5,8


In [9]:
#实现a组和b组，谁比平均分高，谁比平均分低
def diff_mean(s):
    """
        返回数据与均值的差值，s传入的是某一个分组
    """
    return s - s.mean()


# groupby 的 transform 方法用于对分组后的数据进行转换操作，返回一个与原始数据形状相同的对象。transform 通常用于在分组后对每个组内的数据进行某种计算，并将结果广播回原始数据的形状。
print(df_obj.groupby('key1').transform(diff_mean))

   data1     data2
0   -2.8 -1.800000
1   -1.0  2.666667
2   -1.8  1.200000
3    4.0  0.666667
4    2.2 -3.800000
5   -3.0 -3.333333
6    2.2  1.200000
7    0.2  3.200000


### homework2：练习merge

In [10]:
import pandas as pd
import numpy as np

df_obj1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                        'data1': np.random.randint(0, 10, 7)})
df_obj2 = pd.DataFrame({'key': ['a', 'b', 'd'],
                        'data2': np.random.randint(0, 10, 3)})

print(df_obj1)
print('-' * 50)
print(df_obj2)

  key  data1
0   b      9
1   b      7
2   a      1
3   c      2
4   a      5
5   a      6
6   b      4
--------------------------------------------------
  key  data2
0   a      3
1   b      2
2   d      8


In [11]:
#默认连接使用相同的列名，连接方式是内连接
pd.merge(df_obj1, df_obj2)

,key,data1,data2
0,b,9,2
1,b,7,2
2,a,1,3
3,a,5,3
4,a,6,3
5,b,4,2


In [12]:
#左df和右df都拿索引连接
pd.merge(df_obj1, df_obj2, left_index=True, right_index=True)

,key_x,data1,key_y,data2
0,b,9,a,3
1,b,7,b,2
2,a,1,d,8


In [13]:
#左表和右表都拿key列来连接
pd.merge(df_obj1, df_obj2, on='key')

,key,data1,data2
0,b,9,2
1,b,7,2
2,a,1,3
3,a,5,3
4,a,6,3
5,b,4,2


In [14]:
# 更改列名
df_obj1 = df_obj1.rename(columns={'key': 'key1'})
df_obj2 = df_obj2.rename(columns={'key': 'key2'})

In [15]:
df_obj1

,key1,data1
0,b,9
1,b,7
2,a,1
3,c,2
4,a,5
5,a,6
6,b,4


In [16]:
df_obj2

,key2,data2
0,a,3
1,b,2
2,d,8


In [17]:
#左表以key1来连接，右表以key2来连接
pd.merge(df_obj1, df_obj2, left_on='key1', right_on='key2')

,key1,data1,key2,data2
0,b,9,b,2
1,b,7,b,2
2,a,1,a,3
3,a,5,a,3
4,a,6,a,3
5,b,4,b,2


In [18]:
#全外连接
pd.merge(df_obj1, df_obj2, left_on='key1', right_on='key2', how='outer')  #全外连接

,key1,data1,key2,data2
0,a,1.0,a,3.0
1,a,5.0,a,3.0
2,a,6.0,a,3.0
3,b,9.0,b,2.0
4,b,7.0,b,2.0
5,b,4.0,b,2.0
6,c,2.0,NaN,NaN
7,NaN,NaN,d,8.0


In [19]:
# left等价于数据库的left  join
pd.merge(df_obj1, df_obj2, left_on='key1', right_on='key2', how='left')

,key1,data1,key2,data2
0,b,9,b,2.0
1,b,7,b,2.0
2,a,1,a,3.0
3,c,2,NaN,NaN
4,a,5,a,3.0
5,a,6,a,3.0
6,b,4,b,2.0


In [20]:
# right等价于数据库的right join
pd.merge(df_obj1, df_obj2, left_on='key1', right_on='key2', how='right')

,key1,data1,key2,data2
0,a,1.0,a,3
1,a,5.0,a,3
2,a,6.0,a,3
3,b,9.0,b,2
4,b,7.0,b,2
5,b,4.0,b,2
6,NaN,NaN,d,8


In [24]:
# 处理重复列名
df_obj1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                        'data': np.random.randint(0, 10, 7)})
df_obj2 = pd.DataFrame({'key': ['a', 'b', 'd'],
                        'data': np.random.randint(0, 10, 3)})

In [25]:
df_obj1

,key,data
0,b,9
1,b,3
2,a,8
3,c,4
4,a,4
5,a,1
6,b,1


In [26]:
df_obj2

,key,data
0,a,9
1,b,7
2,d,4


In [27]:
#给相同的数据列添加后缀
print(pd.merge(df_obj1, df_obj2, on='key', suffixes=('_left', '_right')))

  key  data_left  data_right
0   b          9           7
1   b          3           7
2   a          8           9
3   a          4           9
4   a          1           9
5   b          1           7


In [29]:
df_obj1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                        'data1': np.random.randint(0, 10, 7)})
df_obj2 = pd.DataFrame({'data2': np.random.randint(0, 10, 3)}, index=['a', 'b', 'd'])

In [30]:
df_obj1

,key,data1
0,b,7
1,b,8
2,a,2
3,c,8
4,a,4
5,a,9
6,b,7


In [31]:
df_obj2

,data2
a,5
b,1
d,8


In [36]:
# 代码 1 以 df_obj1 的 'key' 列和 df_obj2 的索引为键，合并两个 DataFrame。
pd.merge(df_obj1, df_obj2, left_on='key', right_index=True)

,key,data1,data2
0,b,7,1
1,b,8,1
2,a,2,5
4,a,4,5
5,a,9,5
6,b,7,1


In [35]:
# 代码 2 以 df_obj2 的索引和 df_obj1 的 'key' 列为键，合并两个 DataFrame。
pd.merge(df_obj2, df_obj1, left_index=True, right_on='key')

,data2,key,data1
2,5,a,2
4,5,a,4
5,5,a,9
0,1,b,7
1,1,b,8
6,1,b,7


上面两个代码的区别是：
代码 1 的结果以 df_obj1 为基础，保留 df_obj1 的所有列。
代码 2 的结果以 df_obj2 为基础，保留 df_obj2 的所有列。

### homework3：练习重复值处理

In [37]:
import pandas as pd
import numpy as np

df_obj = pd.DataFrame({'data1': ['a'] * 4 + ['b'] * 4,
                       'data2': np.random.randint(0, 4, 8)})
print(df_obj)

  data1  data2
0     a      3
1     a      1
2     a      1
3     a      2
4     b      2
5     b      3
6     b      0
7     b      0


In [38]:
# Pandas 中用于检测 DataFrame 中重复行的方法。它会返回一个布尔类型的 Series，表示每一行是否是重复行（True 表示重复，False 表示不重复）
print(df_obj.duplicated())

0    False
1    False
2     True
3    False
4    False
5    False
6    False
7     True
dtype: bool


In [39]:
df_obj[~df_obj.duplicated()]  #取出不重复行

,data1,data2
0,a,3
1,a,1
3,a,2
4,b,2
5,b,3
6,b,0


In [40]:
#按照某一列去重
print(df_obj.duplicated('data2'))

0    False
1    False
2     True
3    False
4     True
5     True
6    False
7     True
dtype: bool


In [41]:
df_obj1 = pd.DataFrame({'data1': [np.nan] * 4,
                        'data2': list('1235')})
df_obj1

,data1,data2
0,NaN,1
1,NaN,2
2,NaN,3
3,NaN,5


In [44]:
#在pd的duplicated认为空值和空值相等的
df_obj1.duplicated('data1')

0    False
1     True
2     True
3     True
dtype: bool

In [45]:
print(df_obj1.drop_duplicates('data1'))  #删除重复行

   data1 data2
0    NaN     1


In [46]:
print(df_obj.drop_duplicates())  #删除重复行

  data1  data2
0     a      3
1     a      1
3     a      2
4     b      2
5     b      3
6     b      0


In [48]:
print(df_obj.drop_duplicates('data2'))

  data1  data2
0     a      3
1     a      1
3     a      2
6     b      0


In [49]:
#如果要在原有的df上去重，需要加inplace=True
df_obj

,data1,data2
0,a,3
1,a,1
2,a,1
3,a,2
4,b,2
5,b,3
6,b,0
7,b,0


In [51]:
#map与 applymap一样，但是map只能用于series，applymap只能用于df
ser_obj = pd.Series(np.random.randint(0, 10, 10))  #series 用map
print(ser_obj)

print(ser_obj.map(lambda x: x ** 2))

0    1
1    1
2    1
3    7
4    7
5    7
6    9
7    5
8    4
9    4
dtype: int32
0     1
1     1
2     1
3    49
4    49
5    49
6    81
7    25
8    16
9    16
dtype: int64


In [52]:
#异常值手动替换
ser_obj = pd.Series(np.arange(10), index=range(3, 13))
ser_obj

3     0
4     1
5     2
6     3
7     4
8     5
9     6
10    7
11    8
12    9
dtype: int64

In [53]:
# 单个值替换单个值
print(ser_obj.replace(1, -100))

3       0
4    -100
5       2
6       3
7       4
8       5
9       6
10      7
11      8
12      9
dtype: int64


In [54]:
# 多个值替换一个值
print(ser_obj.replace(range(6, 9), -100))

3       0
4       1
5       2
6       3
7       4
8       5
9    -100
10   -100
11   -100
12      9
dtype: int64


In [55]:
# 多个值替换多个值
print(ser_obj.replace([4, 7], [-100, -200]))

3       0
4       1
5       2
6       3
7    -100
8       5
9       6
10   -200
11      8
12      9
dtype: int64


In [56]:
df = pd.DataFrame({'A': [0, 1, 2, 3, 4],
                   'B': [5, 6, 7, 8, 9],
                   'C': ['a', 'b', 'ac', 'd', 'e']})
df

,A,B,C
0,0,5,a
1,1,6,b
2,2,7,ac
3,3,8,d
4,4,9,e


In [57]:
#正则表达式替换
"""
to_replace=r'^a':
这是一个正则表达式，^a 表示匹配以字母 'a' 开头的字符串。
^ 是正则表达式中的锚点，表示字符串的开头。

value=100:
将匹配到的值替换为 100。

regex=True:
表示 to_replace 参数是一个正则表达式。
"""
df.replace(to_replace=r'^a', value=100, regex=True)

,A,B,C
0,0,5,100
1,1,6,b
2,2,7,100
3,3,8,d
4,4,9,e


In [58]:
df.dtypes

A     int64
B     int64
C    object
dtype: object